## Agents Demystified - part 3

In this installment we'll let our simple agent loose on portal data by revisiting the introductory tutorial on [using the portal and Pandas](../PortalAndPandas/introduction.ipynb). In doing so, we'll also hit some of the limits of our design. 

First we will use the agent on log data from ERDC and run some examples, becoming aware of the limitations of our design choices in this tutorial, before concluding with a summary of the main takeaways and a discussion of what would be required components of a "real" agentic system. Hopefully, by then, agents in the wild will make a lot more sense as you have enough experience to push further by yourself.

### Getting the DataportalClient

First we need to get the DataportalClient module, and then set up the portal token but this time we'll pass it in the environment rather than pasting it into a script. 

In [ ]:
!pip install -q git+https://github.com/wara-ops/DataportalClient.git

In [ ]:
token = "" # <-- Paste your portal token between the quotes

import os
os.environ['DATAPORTAL_CLIENT_TOKEN'] = token

### Getting the agent and LLM-as-a-Service

In [ ]:
!pip -q install ollama tavily

In [ ]:
from helpers.agents2 import Agent
from helpers.agents3 import execute_script

In [ ]:
# Host and model definitions
OLLAMA_HOST = 'http://10.129.20.4:9090'
OLLAMA_MODEL = 'gemma3:27b-it-qat'

### Sanity test

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[execute_script])
print(agent.task("What time is it?"))

### Accessing portal data

This is where we hit the first limitation in the current design. Since our agent relies on `execute_script` (which runs a separate process) to evaluate any generated code we really can't keep any state (even in the agent's internal loop) except what is contained in the (text) message history and, possibly more limiting, any (portal related) code that needs to be used by scripts has to be _imported_ by the script as we can't simply pass python objects to `execute_script`.   

Thus, to simplify things we'll place a portal "helper" function the working director that can be imported by the script. Running the cell below will write the python code to `xerces.py` in the working directory.

In [ ]:
%%bash
"""
cat << EOF > work/xerces.py
from dataportal import DataportalClient
import pandas


A simple helper function to access a (hard-coded) log from the portal

def get_log() -> pandas.DataFrame:
    """
    Return a Xerces log file as a pandas DataFrame.
    Xerces is a cloud service, built using OpenStack, consisting of subsystem modules. 
    The log entries are tagged with the name of the emitting module.

    The modules are:
    - 'Nova' handles virtual machines.
    - 'Neutron' provides "networking-as-a-service".
    - 'Swift' provides an object storage.
    - 'Cinder' offers persistent block storage.
    - 'Horizon' is the GUI for OpenStack.
    - 'Keystone' provides identity services.
    - 'Glance' handle discovery and retrieve of virtual machine images.
    - 'Heat' orchestrates multiple composite cloud applications.

    Args:
        None 

    Returns:
        pandas.DataFrame: the Xerces log file as a pandas DataFrame
    """

    client = DataportalClient()
    client.fromDataset('ERDClogs-parsed')
    return client.getData(fileID=36666)
    
EOF
"""

### Analyzing the logs

Now we're ready to go! The first thing is to inform the agent how to access the log data, and get some feedback into the message history:

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[execute_script])
print(agent.task("You have access to logs from the cloud service 'Xerces' by importing the function 'get_log' from 'xerces.py' in your working directory. After importing it, you can retrieve documentation for the function using 'help(get_log)'. Give me a terse summary of the 'Xerces' service subsystems."))

In [ ]:
# print(agent.message_history())

Now we can e.g. ask questions on what was logged …

In [ ]:
print(agent.task("Get info on the log dataframe and list the column labels."))

… and quickly scan the log for signs of trouble …

In [ ]:
print(agent.task("Great! Get all 'log_level' entries and make a list of all types. Only list each type once."))

In [ ]:
print(agent.task("The naming appears inconsistent. Coalesce 'log_level' types as the uppercased long form, e.g. 'warn', 'warning', 'Warning' should all be 'WARNING'. Get all 'log_level' entries and make a list of all types."))

### Hitting another limit

About this far into the conversation things usually start to get shaky. By now the message history is quite long (the system prompt plus all internal conversations plus our queries) and it is fed to the LLM _on every turn_ which means we are taxing the LLM's context window as well as the physical memory required by the LLM to process such an extensive string of tokens. Every answer tend to take longer and longer to complete. Furthermore, if the message history grows beyond the allowed context window size, the input is simply truncated and the agent is not likely to respond sensibly.

To see the message history (sans system prompt):

In [ ]:
print(agent.message_history())

### Band aid…
Starting a new agent solves the problem with the context size:

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[execute_script])
instructions = """
You have access to logs from the cloud service 'Xerces' by importing the function 'get_log' from 'xerces.py' in your working directory. 
After importing it, you can retrieve documentation for the function using 'help(get_log)'. 
Unify 'log_level' types as the uppercased long form, e.g. 'warn', 'warning', 'Warning' should all be 'WARNING'.
Modify the dataframe accordingly and save it in CSV format as 'cleaned_log.csv'
"""
print(agent.task(instructions))

You can open the resulting file ['work/cleaned_log.csv'](work/cleaned_log.csv) and verify the fixes.

### Visualizing log events

Just as in the previous part, the agent is capable of visualizing data as per your request:

In [ ]:
agent = Agent(OLLAMA_HOST, OLLAMA_MODEL, tools=[execute_script])
instructions = """
You have access to logs from the cloud service 'Xerces' by importing the function 'get_log' from 'xerces.py' in your working directory. 
After importing it, you can retrieve documentation for the function using 'help(get_log)'. 
Display a bar chart over the number of log entries per module ('Logger')
"""

print(agent.task(instructions))

In [ ]:
# Continued
print(agent.task("Using the log dataframe, display a bar chart over the number of log entries per type ('log_level') for entries whose 'Logger' column is tagged 'openstack.neutron'."))

## Summary and dicussion

So, in this three-part tutorial we have tried to show that AI agents are not magical by building a simple, yet capable, agent around a vanilla LLM using nothing but plain python.

As always, the devil is in the details, and the simple design hit a number of limitations. The important take-away is that those limitations were not due to the core concept, but necessary to keep this tutorial short and to the point. 

Finally, an example of how a full implementation of an AI agent could be structured (hopefully this picture makes sense after following the tuorials):

```mermaid
flowchart LR
    user_prompt("User prompt"):::start_stop
    agent(("Agent")):::model
    llm["LLM"]
    hist["Curated history"]
    long["Long term memory"]
    short["Short term memory"]
    rag["RAG"]
    tools["Tools"]
    work_area["Work area"]
    state["State"]
    

    user_prompt --> agent --> llm
    llm --> parsing --> agent

    subgraph Conversation History
        hist <--> long
        hist <--> short
    end
    agent <--> hist

    rag <--> agent

    subgraph Tooling
        tools <--> work_area <--> state
    end
    agent --> tools
    state --> agent

    classDef start_stop fill: #9f6, stroke: #333, stroke-width:2px;
    classDef model fill: #f96, stroke: #333, stroke-width:1px;

```